## Lite LLM 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from agents import Agent, Runner
import nest_asyncio
nest_asyncio.apply()

# Access your API key
if not os.getenv("OPENAI_API_KEY"):
   raise ValueError("OPENAI_API_KEY not found in .env file")

In [3]:
from litellm import completion
import pprint
import textwrap
from pydantic import BaseModel
from typing import Optional

#### call openai with formated output 

In [37]:
# Define Pydantic model for structured weather response
class WeatherResponse(BaseModel):
    location: str
    temperature: Optional[str] = None
    conditions: Optional[str] = None
    humidity: Optional[str] = None
    wind: Optional[str] = None
    summary: str

## Call 4o mini with structured output
messages=[{"role": "system", "content": "You are a helpful weather assistant. Respond with structured weather information."},
          {"role": "user", "content": "what's the weather in SF"}]

response = completion(
    messages=messages,
    model='gpt-4o-mini',
    max_completion_tokens=100,  ## equals to max output tokens
    response_format=WeatherResponse  # Use Pydantic model for structured output
)

# Parse the structured response
weather_data = response['choices'][0]['message']['content']
print("Structured Weather Response:")
print(weather_data)

Structured Weather Response:
{"location":"San Francisco, CA","temperature":"64°F","conditions":"Partly cloudy","humidity":"70%","wind":"5 mph","summary":"A mild day with some clouds."}


#### call reasoning models 

In [15]:
## call thinking models 
# Call o1-mini thinking model to get both reasoning and final answer
messages = [{"role": "user", "content": "Explain the concept of recursion in programming with a simple example.Please think carefully and provide a detailed explanation."}]

# Make the completion call to o1-mini
response = completion(
    model="anthropic/claude-3-7-sonnet-20250219",
    messages=messages,
    reasoning_effort="low", 
    )
# Check if reasoning content exists and print accordingly
if response['choices'][0]['message'].get('reasoning_content'):
    print("Reasoning Content:")
    print(response['choices'][0]['message']['reasoning_content'])
    print("\nFinal Answer:")
    print(response.choices[0].message.content[:100])
else:
    print("Raw Response:")
    print(response.choices[0].message.content[:100])



Reasoning Content:
Recursion is a fundamental concept in programming where a function calls itself to solve a problem. Let me plan a thorough explanation with a simple example.

Key points to cover:
1. Definition of recursion
2. How recursion works conceptually
3. Key components of a recursive function (base case and recursive case)
4. A simple example (factorial or fibonacci are common)
5. Tracing through the example step by step
6. Advantages and disadvantages of recursion
7. Common pitfalls and considerations

For the example, I'll use calculating factorial, which is a classic and intuitive example of recursion. Factorial of n (written as n!) is the product of all positive integers less than or equal to n.

Example: 5! = 5 × 4 × 3 × 2 × 1 = 120

This is a good example because:
- It has a clear mathematical definition
- It can be expressed recursively: n! = n × (n-1)!
- It has a simple base case: 0! = 1 or 1! = 1
- The recursion depth is manageable for examples

Final Answer:
# Recur

#### Call gogole gemini 2.5 

In [17]:
response = completion(
  model="gemini/gemini-2.5-flash-preview-04-17",
  messages=[{"role": "user", "content": "What is the capital of France?"}],
  thinking={"type": "enabled", "budget_tokens": 1024},
)

print(response.choices[0].message.content)

The capital of France is **Paris**.


#### call any other openai competable api  [netmind Error]

In [28]:
#### there is a problem with server setup 
response = completion(
            model="Qwen/QwQ-32B",
            messages=messages,
            api_base="https://api.netmind.ai/inference-api/openai/v1",
            api_key=os.getenv("NETMIND_API_KEY"),
            temperature=0.2,
            max_tokens=2000)


Provider List: https://docs.litellm.ai/docs/providers



BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=Qwen/QwQ-32B
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [45]:
import openai
client = openai.OpenAI(
    api_key=os.getenv("NETMIND_API_KEY"),            # pass litellm proxy key, if you're using virtual keys
    base_url="https://api.netmind.ai/inference-api/openai/v1" # litellm-proxy-base url
)

response = client.chat.completions.create(
    model="Qwen/QwQ-32B",
    messages = [
        {
            "role": "user",
            "content": "what llm are you"
        }
    ],
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content
print('Reasoning Content:')
print(reasoning_content)
print('Content:')
print(content)

Reasoning Content:
Okay, the user is asking "what llm are you". Let me break this down. LLM stands for Large Language Model, so they're asking which one I am.

First, I need to confirm that they're referring to my model type. Since I'm Qwen, I should explain that I am a large language model developed by Alibaba Cloud. 

They might want to know specifics like my architecture, training data, or capabilities. I should mention that I'm based on the Qwen series, which uses advanced transformer architecture. 

Also, maybe they're curious about my training data. I should state that it's extensive and comes from Alibaba Group's internal historical accumulation, but not specify exact dates or sources due to confidentiality.

I should highlight my features, like handling various tasks, multi-language support, code generation, etc. But keep it concise. 

Wait, should I mention my parameters? The user didn't ask for that specifically, but it's part of being an LLM. Maybe include it briefly. 

Also

## Other useages 
- https://docs.litellm.ai/docs/providers/gemini